# Reading a NetCDF file

First query data through `openeo` and save it locally:

In [1]:
import openeo
import xarray
import matplotlib.pyplot as plt

connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc()

s2_cube = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=("2022-07-12", "2022-07-18"),
    spatial_extent={
        "west": -1.23,
        "south": 44.51,
        "east": -1.12,
        "north": 44.59,
        "crs": "EPSG:4326",
    },
)

file_name = "fire_la_teste_all_bands.nc"
s2_cube.download(file_name)

Authenticated using refresh token.


We now have a 17MB `.nc` file on disk.

## Opening it with h5py

List top-level groups/datasets

In [2]:
import h5py

with h5py.File(file_name, 'r') as f:
    print(list(f.keys()))

['t', 'x', 'y', 'crs', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'WVP', 'AOT', 'SCL', 'sunAzimuthAngles', 'sunZenithAngles', 'viewAzimuthMean', 'viewZenithMean', 'CLD', 'SNW']


Exploring the full file and the metadata of individual dataset:

In [3]:
with h5py.File(file_name, 'r') as f:
    dset = f['/t']
    print(dset.shape)
    print(dset[:]) # to print the data inside a given slice
    for key, val in dset.attrs.items():
        print(f"  {key}: {val}")

(2,)
[11880 11885]
  CLASS: b'DIMENSION_SCALE'
  NAME: b't'
  _Netcdf4Dimid: 0
  standard_name: b't'
  long_name: b't'
  units: b'days since 1990-01-01'
  axis: b'T'
  REFERENCE_LIST: [(<HDF5 object reference>, 0) (<HDF5 object reference>, 0)
 (<HDF5 object reference>, 0) (<HDF5 object reference>, 0)
 (<HDF5 object reference>, 0) (<HDF5 object reference>, 0)
 (<HDF5 object reference>, 0) (<HDF5 object reference>, 0)
 (<HDF5 object reference>, 0) (<HDF5 object reference>, 0)
 (<HDF5 object reference>, 0) (<HDF5 object reference>, 0)
 (<HDF5 object reference>, 0) (<HDF5 object reference>, 0)
 (<HDF5 object reference>, 0) (<HDF5 object reference>, 0)
 (<HDF5 object reference>, 0) (<HDF5 object reference>, 0)
 (<HDF5 object reference>, 0) (<HDF5 object reference>, 0)
 (<HDF5 object reference>, 0)]


Resolving references:

In [4]:
import h5py

with h5py.File(file_name, 'r') as f:
    obj_with_refs = f['/t'] 
    ref_list = obj_with_refs.attrs['REFERENCE_LIST']
    # Dereference and print what each reference points to:
    for i, ref in enumerate(ref_list):
        target = f[ref[0]]  # ref is a tuple: (object_reference, 0)
        print(f"Reference {i}: {target.name} - {type(target)}")

Reference 0: /B01 - <class 'h5py._hl.dataset.Dataset'>
Reference 1: /B02 - <class 'h5py._hl.dataset.Dataset'>
Reference 2: /B03 - <class 'h5py._hl.dataset.Dataset'>
Reference 3: /B04 - <class 'h5py._hl.dataset.Dataset'>
Reference 4: /B05 - <class 'h5py._hl.dataset.Dataset'>
Reference 5: /B06 - <class 'h5py._hl.dataset.Dataset'>
Reference 6: /B07 - <class 'h5py._hl.dataset.Dataset'>
Reference 7: /B08 - <class 'h5py._hl.dataset.Dataset'>
Reference 8: /B8A - <class 'h5py._hl.dataset.Dataset'>
Reference 9: /B09 - <class 'h5py._hl.dataset.Dataset'>
Reference 10: /B11 - <class 'h5py._hl.dataset.Dataset'>
Reference 11: /B12 - <class 'h5py._hl.dataset.Dataset'>
Reference 12: /WVP - <class 'h5py._hl.dataset.Dataset'>
Reference 13: /AOT - <class 'h5py._hl.dataset.Dataset'>
Reference 14: /SCL - <class 'h5py._hl.dataset.Dataset'>
Reference 15: /sunAzimuthAngles - <class 'h5py._hl.dataset.Dataset'>
Reference 16: /sunZenithAngles - <class 'h5py._hl.dataset.Dataset'>
Reference 17: /viewAzimuthMean - 

## Using xarray

also requires `netcdf4`

In [5]:
import xarray

ds = xarray.open_dataset(file_name)
print(ds)

<xarray.Dataset> Size: 137MB
Dimensions:           (t: 2, x: 895, y: 909)
Coordinates:
  * t                 (t) datetime64[ns] 16B 2022-07-12 2022-07-17
  * x                 (x) float64 7kB 6.405e+05 6.405e+05 ... 6.494e+05
  * y                 (y) float64 7kB 4.939e+06 4.939e+06 ... 4.93e+06 4.93e+06
Data variables: (12/22)
    crs               |S1 1B ...
    B01               (t, y, x) float32 7MB ...
    B02               (t, y, x) float32 7MB ...
    B03               (t, y, x) float32 7MB ...
    B04               (t, y, x) float32 7MB ...
    B05               (t, y, x) float32 7MB ...
    ...                ...
    sunAzimuthAngles  (t, y, x) float32 7MB ...
    sunZenithAngles   (t, y, x) float32 7MB ...
    viewAzimuthMean   (t, y, x) float32 7MB ...
    viewZenithMean    (t, y, x) float32 7MB ...
    CLD               (t, y, x) float32 7MB ...
    SNW               (t, y, x) float32 7MB ...
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform


In [6]:
print(ds.coords) 
print(ds.data_vars)

Coordinates:
  * t        (t) datetime64[ns] 16B 2022-07-12 2022-07-17
  * x        (x) float64 7kB 6.405e+05 6.405e+05 ... 6.494e+05 6.494e+05
  * y        (y) float64 7kB 4.939e+06 4.939e+06 4.939e+06 ... 4.93e+06 4.93e+06
Data variables:
    crs               |S1 1B ...
    B01               (t, y, x) float32 7MB ...
    B02               (t, y, x) float32 7MB ...
    B03               (t, y, x) float32 7MB ...
    B04               (t, y, x) float32 7MB ...
    B05               (t, y, x) float32 7MB ...
    B06               (t, y, x) float32 7MB ...
    B07               (t, y, x) float32 7MB ...
    B08               (t, y, x) float32 7MB ...
    B8A               (t, y, x) float32 7MB ...
    B09               (t, y, x) float32 7MB ...
    B11               (t, y, x) float32 7MB ...
    B12               (t, y, x) float32 7MB ...
    WVP               (t, y, x) float32 7MB ...
    AOT               (t, y, x) float32 7MB ...
    SCL               (t, y, x) float32 7MB ...
    su

`xarray` can automatically spot coordinates VS datasets!

In [7]:
print(ds["B01"].data)
print(f"The data is loaded as: {type(ds["B01"].data).__name__}")

[[[654. 448. 448. ... 247. 247. 247.]
  [654. 448. 448. ... 247. 247. 247.]
  [654. 448. 448. ... 247. 247. 247.]
  ...
  [220. 213. 213. ...  nan  nan  nan]
  [220. 213. 213. ...  nan  nan  nan]
  [220. 213. 213. ...  nan  nan  nan]]

 [[321. 137. 137. ...  61.  61.  61.]
  [321. 137. 137. ...  61.  61.  61.]
  [321. 137. 137. ...  61.  61.  61.]
  ...
  [200. 196. 196. ...  nan  nan  nan]
  [200. 196. 196. ...  nan  nan  nan]
  [200. 196. 196. ...  nan  nan  nan]]]
The data is loaded as: ndarray


## Lazy loading with xarray

This loads the data as a Dask `Array` instead of a numpy `ndarray`.

In [8]:
ds = xarray.open_dataset(file_name, chunks={})
print(ds.chunks)

Frozen({'t': (1, 1), 'y': (256, 256, 256, 141), 'x': (256, 256, 256, 127)})


In [9]:
print(ds["B01"].data)
print(f"The data is loaded as: {type(ds['B01'].data).__name__}")

dask.array<open_dataset-B01, shape=(2, 909, 895), dtype=float32, chunksize=(1, 256, 256), chunktype=numpy.ndarray>
The data is loaded as: Array


In the lazy loading case, if you want to access the values you need to use `.values` instead of `.data`:

In [10]:
print(ds["B01"].values)
print(f"The data is loaded as: {type(ds["B01"].values).__name__}")

[[[654. 448. 448. ... 247. 247. 247.]
  [654. 448. 448. ... 247. 247. 247.]
  [654. 448. 448. ... 247. 247. 247.]
  ...
  [220. 213. 213. ...  nan  nan  nan]
  [220. 213. 213. ...  nan  nan  nan]
  [220. 213. 213. ...  nan  nan  nan]]

 [[321. 137. 137. ...  61.  61.  61.]
  [321. 137. 137. ...  61.  61.  61.]
  [321. 137. 137. ...  61.  61.  61.]
  ...
  [200. 196. 196. ...  nan  nan  nan]
  [200. 196. 196. ...  nan  nan  nan]
  [200. 196. 196. ...  nan  nan  nan]]]
The data is loaded as: ndarray
